## docu

- [httpx quick start](https://www.python-httpx.org/quickstart/)

### get vs post

attr | get | post
--|--|--
BACK button/Reload | harmless | Data will be re-submitted (the browser should alert the user that the data are about to be re-submitted)
Bookmarked | Can be bookmarked | Cannot be bookmarked
Cached | Can be cached | Not cached
Encoding type | application/x-www-form-urlencoded | application/x-www-form-urlencoded or multipart/form-data. Use multipart encoding for binary data
History | Parameters remain in browser history | Parameters are not saved in browser history
Restrictions on data length | Yes, when sending data, the GET method adds the data to the URL; and the length of a URL is limited (maximum URL length is 2048 characters) | No restrictions
Restrictions on data type | Only ASCII characters allowed | No restrictions. Binary data is also allowed
Security | GET is less secure compared to POST because data sent is part of the URL. Never use GET when sending passwords or other sensitive information! | POST is a little safer than GET because the parameters are not stored in browser history or in web server logs
Visibility | Data is visible to everyone in the URL | Data is not displayed in the URL


### post vs put

- put
    - are idempotent: calling the same PUT request multiple times will always produce the same result
- post
    - calling a POST request repeatedly have side effects of creating the same resource multiple times

## Authentication

```bash
echo "username:password"|base64
echo "dXNlcm5hbWU6cGFzc3dvcmQK"|base64 -d
```

### status code 200

- 200 is ok for get request. 
- 201 is created with post request

### code 300

- 302 is redirect

### code 400

- 401 is unauthorized. to access data without the correct credentials
- 404 is page not found
- 409 is already exists. to create an existing entity

## code

In [45]:
import httpx
import re
import json
import base64

In [2]:
import lxml.html

In [2]:
URLS = [
        'https://yandex.ru/',
        'https://weather.data-analyst.praktikum-services.ru/v1/forecast',
        'https://fakestoreapi.com/products',
        'https://store.steampowered.com/explore/new/'
        ]
PARAM = {'page': '4'}
params1 = {
        'city' : 'Moscow',
        'hours' : True
    }

In [ ]:
api_set_list = []
# url, data, params

In [59]:
def jprint(obj):
    # create a formatted string of the Python JSON object
    text = json.dumps(obj, sort_keys=True, indent=4)
    print(text)

In [2]:
res = httpx.get(url=URLS[2], params=params1) 

f'code {res.status_code} and encoding {res.encoding}', \
    res.url

NameError: name 'httpx' is not defined

### Redirection and History

In [24]:
res = httpx.get('http://github.com/')  # status code 301

In [25]:
res.next_request

<Request('GET', 'https://github.com/')>

In [20]:
res = httpx.get('http://github.com/', follow_redirects=True)  # status code 200

In [22]:
res.history

[<Response [301 Moved Permanently]>]

In [4]:
res = httpx.get(url=URLS[4-1])

In [ ]:
res.text

In [ ]:
res.json()

In [ ]:
jprint(res.json())

In [ ]:
for prod in res.json():
    print(prod['price'])

In [ ]:
json.loads(res.text)

In [5]:
html = lxml.html.fromstring(res.content)

In [6]:
new_releases = html.xpath('//div[@id="tab_newreleases_content"]')[0]
titles = new_releases.xpath('.//div[@class="tab_item_name"]/text()')
prices = new_releases.xpath('.//div[@class="discount_final_price"]/text()')

output = []

for info in zip(titles, prices):
    games = {}
    games['title'] = info[0]
    games['price'] = info[1]
    output.append(games)

output[:3]

[{'title': 'Atari 50: The Anniversary Celebration', 'price': '$18.49'},
 {'title': 'Lords and Villeins', 'price': '$16.49'},
 {'title': 'Among Us VR', 'price': '$6.29'}]

## POST testing

- are never cached
- do not remain in the browser history
- cannot be bookmarked
- have no restrictions on data length

In [12]:
payload = {
    'title': 'new product',
    'price': 13.5,
    'description': 'test description',
    'image': '',
    'category': 'electronic'
}

In [15]:
res_post = httpx.post(url=URLS[2], data=payload)

In [18]:
res_post.json()

{'id': 21,
 'title': 'new product',
 'price': '13.5',
 'description': 'test description',
 'image': '',
 'category': 'electronic'}

### Authentication

In [31]:
auth = httpx.DigestAuth("my_user", "password123")
auth._username

b'my_user'

In [29]:


res = httpx.get("https://example.com", auth=auth)
res


<Response [200 OK]>

In [52]:
with httpx.Client(auth=('tom', 'mot123')) as client:
    r = client.get('https://example.com', auth=('alice', 'ecila123'))
auth = r.request.headers['Authorization'].partition(' ')
auth

('Basic', ' ', 'YWxpY2U6ZWNpbGExMjM=')

In [53]:
_, _, auth = r.request.headers['Authorization'].partition(' ')

In [54]:
base64.b64decode(auth)

b'alice:ecila123'

### ssl

In [ ]:
httpx.get("https://localhost:8000")

In [ ]:
httpx.get("https://localhost:8000", verify="/tmp/client.pem")

In [ ]:
# ConnectError: [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:997)
httpx.get('https://expired.badssl.com/')

In [65]:
# verify ='/path/to/certfile'
# cert=('/path/client.cert', '/path/client.key'))
ssl_bypass = httpx.get('https://expired.badssl.com/', verify = False)
ssl_bypass

<Response [200 OK]>

In [32]:
try:
    response = httpx.get("https://www.example.com/")
except httpx.RequestError as exc:
    print(f"An error occurred while requesting {exc.request.url!r}.")

In [33]:
response = httpx.get("https://www.example.com/")
try:
    response.raise_for_status()
except httpx.HTTPStatusError as exc:
    print(f"Error response {exc.response.status_code} while requesting {exc.request.url!r}.")


In [34]:
try:
    response = httpx.get("https://www.example.com/")
    response.raise_for_status()
except httpx.HTTPError as exc:
    print(f"Error while requesting {exc.request.url!r}.")

In [35]:
try:
    response = httpx.get("https://www.example.com/")
    response.raise_for_status()
except httpx.RequestError as exc:
    print(f"An error occurred while requesting {exc.request.url!r}.")
except httpx.HTTPStatusError as exc:
    print(f"Error response {exc.response.status_code} while requesting {exc.request.url!r}.")


## Get

- /test/demo_form.php?name1=value1&name2=value2
- can be cached
- remain in the browser history
- can be bookmarked
- should never be used when dealing with sensitive data
- have length restrictions
- are only used to request data (not modify)

In [36]:
with httpx.Client() as client:
    r = client.get('https://example.com')
r

<Response [200 OK]>

In [37]:
with httpx.Client() as client:
    headers = {'X-Custom': 'value'}
    r = client.get('https://example.com', headers=headers)
r.request.headers['X-Custom']

'value'

In [38]:
url = 'http://httpbin.org/headers'
headers = {'user-agent': 'my-app/0.0.1'}
with httpx.Client(headers=headers) as client:
    r = client.get(url)
r.json()['headers']['User-Agent']

'my-app/0.0.1'

In [44]:
headers = {'X-Auth': 'from-client'}
params = {'client_id': 'client1'}
with httpx.Client(headers=headers, params=params) as client:
    headers = {'X-Custom': 'from-request'}
    params = {'request_id': 'request1'}
    r = client.get('https://example.com', headers=headers, params=params)

print(f'''
{r.request.url} 
{r.request.headers['X-Auth']} 
{r.request.headers['X-Custom']}
''')


https://example.com?client_id=client1&request_id=request1 
from-client 
from-request



### streaming responses

In [ ]:
with httpx.stream("GET", "https://www.example.com") as r:
    for data in r.iter_bytes():
        print(data)

## delete testing

In [19]:
res_del = httpx.delete('https://fakestoreapi.com/products/21')

In [21]:
res_del.json()

## Event Hooks

In [56]:
def log_request(request):
    print(f"Request event hook: {request.method} {request.url} - Waiting for response")

def log_response(response):
    request = response.request
    print(f"Response event hook: {request.method} {request.url} - Status {response.status_code}")

client = httpx.Client(event_hooks={'request': [log_request], 'response': [log_response]})


## JS

In [ ]:
fetch('https://fakestoreapi.com/products',
  ).then(res =>{
    if (res.ok){
    return res.json()
}
}).then(response=>{
    console.log(response)
}).catch(err => console.log(err))

In [1]:
%%javascript
fetch('https://fakestoreapi.com/products',
  ).then(res =>{
    if (res.ok){
    return res.json()
}
}).then(response=>{
    response.map(data =>{
        console.log(data.price)
    })
    // console.log(response)
}).catch(err => console.log(err))

<IPython.core.display.Javascript object>

In [ ]:
// Specify the payload 
let payload = {
    title: 'new product',
    price: 13.5,
    description: 'test description',
    image: '',
    category: 'electronic'
}

fetch('https://fakestoreapi.com/products',
{
    method: "Post",
    headers:{
        'Content-Type': 'application/json'  
    },
    body: JSON.stringify(payload) //convert the payload to JSON 
}
  ).then(res =>{
      if (res.ok){
    console.log(res.status)
    return res.json()
}
}).then(response => {
    console.log(response)
}).catch(err => console.log(err))

In [ ]:
fetch('https://fakestoreapi.com/products/19',
{
    method: "Put",
    headers:{
        'Content-Type': 'application/json'  
    },
    body: JSON.stringify(payload) //convert the payload into JSON
}
  ).then(res =>{
      if (res.ok){
    console.log(res.status)
    return res.json()
}
}).then(response => {
    console.log(response)
}).catch(err => console.log(err))

In [ ]:
fetch('https://fakestoreapi.com/products/19',
{
    method: "Delete",
    headers:{
        'Content-Type': 'application/json'  
    }
}
  ).then(res =>{
      if (res.ok){
    console.log(res.status)
    return res.json()
}
}).then(response => {
    console.log(response)
}).catch(err => console.log(err))